# Generating Star Trek Dialogue with Transformers
## Jared Talon Holton

In [ ]:
f = open("all_scripts_raw.json", "r")
data = json.load(f)

split_words = re.compile(r'[\s\[\]\(\),.?!:]')
split_sentence = re.compile(r'(?<=[?!.])(?![\d]+)')

In [ ]:
sentences = {}
raw_sentences = []
words = []
print("building sentence corpus...")
for series in df_scenes:
    sentences[series] = []
    for ep in df_scenes[series]:
        if type(ep) != list:
            continue

        for scene in ep:
            if type(scene) != str:
                continue
            
            for s in split_sentence.split(scene.strip().lower()):
                raw_sentences.append(s)     
               
    print(f"Finished loading {series}")

print("Done!") 

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '<pad>'})
vocab_length = tokenizer.vocab_size


def tokenize(s, max_len=128):
    tok = tokenizer.encode(s)
    return tf.constant(tok, dtype=tf.int32)

raw_gpt_tokens = []
for s in tqdm.tqdm(raw_sentences):
    raw_gpt_tokens.append(tokenize(s))

In [ ]:
combined_sentences = []
scratch = []
for s in tqdm.tqdm(raw_gpt_tokens):
    if (len(scratch) + len(s)) < 128:
        scratch = tf.concat([scratch, s], axis=0)
        # print(scratch)
    else:
        scratch = tf.concat(
                [scratch, [tokenizer.pad_token_id]*(128 - len(scratch))], axis=0)
        s = tf.concat([s, [tokenizer.pad_token_id]*((128 - len(s)))], axis=0)
        combined_sentences.append((scratch, s))
        scratch = []
        # break

def scaled_dot_product_attention(q, k, v, mask=None):
        """Calculate the attention weights.
        q, k, v must have matching leading dimensions.
        k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
        The mask has different shapes depending on its type(padding or look ahead)
        but it must be broadcastable for addition.

        Args:
            q: query shape == (..., seq_len_q, depth)
            k: key shape == (..., seq_len_k, depth)
            v: value shape == (..., seq_len_v, depth_v)
            mask: Float tensor with shape broadcastable
                to (..., seq_len_q, seq_len_k). Defaults to None.

        Returns:
            output, attention_weights
        """
        
        matmul_qk = tf.matmul(q, k, transpose_b=True)

        # scale matmul_qk
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

        # add the mask to the scaled tensor.
        if mask is not None:
            scaled_attention_logits += (mask * -1e9)

        # softmax is normalized on the last axis (seq_len_k) so that the scores
        # add up to 1.
        attention_weights = tf.nn.softmax(
            scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

        output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

        return output, attention_weights

In [ ]:
def offsetElu(x):
        return tf.nn.elu(x) + 1

    def build_linear_attention(feature_map=None):
        if feature_map is None:
            feature_map = AttentionUtils.offsetElu

        def lin_attention(q, k, v, mask=None):
            """Calculate the attention weights.
                q, k, v must have matching leading dimensions.
                k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
                The mask has different shapes depending on its type(padding or look ahead)
                but it must be broadcastable for addition.

                Args:
                    q: query shape == (..., seq_len_q, depth)
                    k: key shape == (..., seq_len_k, depth)
                    v: value shape == (..., seq_len_v, depth_v)
                    mask: Float tensor with shape broadcastable
                        to (..., seq_len_q, seq_len_k). Defaults to None.

                Returns:
                    output, attention_weights
                """

            fQ = feature_map(q)
            fK = feature_map(k)

            Z = 1/tf.einsum('bhsi,bhsi->bhsi', q, tf.cumsum(k, axis=1))
            S = tf.cumsum(tf.einsum('bhsi,bhsi->bhsi', q, v), axis=1)

            Vp = tf.multiply(q, S)
            Vp = tf.realdiv(Vp, Z)

            return Vp, None
        return lin_attention